<a href="https://colab.research.google.com/github/bsalami-092/Blessing-ML-Zoomcamp-2025/blob/main/Flood_Risk_Index_of_Lagos_State.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building a Flood Risk Data Pipeline for Lagos

In [ ]:
!pip install geemap earthengine.api rasterio folium contextily shapely fiona

In [ ]:
# Import required libraries
import ee
import numpy as np
import rasterio
import folium
import contextily as cx
import shapely.geometry
import fiona
import geemap
import matplotlib.pyplot as plt

Initialize and Authenticate  Google Earth Engine

In [ ]:
# Authenticate GEE (only needed once per session)
ee.Authenticate()

# Initialize GEE
ee.Initialize(project='geo-coding-475422')

Define the Area of Interest(Lagos, Nigeria)

In [ ]:
# Define area of interest using minimum and maximum coordinates
aoi = ee.Geometry.Rectangle((2.926, 6.358, 3.734, 6.673))  #(minlon, minlat, maxlon, maxlat)


# Visualize the area
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.addLayer(aoi, {}, 'Lagos Boundary')
Map

Acquire and preprocess spatial data. To do that,  we need four essential datasets from GEE
* Digital Elevation Model(DEM) for terrain analysis
* Rainfall data to assess precipitation levels
* Land Use/Land Cover (LULC) for surfacr classification
* Population data to evaluate human exposure to flood s


Load Digital Elevation Model (DEM)

In [ ]:
# Load SRTM DEM dataset (30m resolution) to extract elevation and identify low-lying areas prone to flood
dem = ee.Image('USGS/SRTMGL1_003').clip(aoi)

# Define visualization parameters
dem_vis = {
    'min': 0,
    'max': 100,
    'palette': ['red', 'blue', 'yellow', 'green']
}

# Add DEM layer to map
Map.addLayer(dem, dem_vis, 'Digital Elevation Model (DEM)')
Map

In [ ]:
# Download DEM as GeoTIFF
geemap.ee_export_image(dem, filename='dem_lagos.tif', scale=30, region=aoi, file_per_band=False)

Load Rainfall Data (CHIRPS Precipitation)

In [ ]:
# Load CHIRPS Rainfall dataset(Climate Hazrd Group Infared Precipitation) to assess flood risk due
rainfall = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD') \
.filterBounds(aoi) \
.filterDate('2020-01-01', '2023-12-31') \
.mean() \
.clip(aoi)

# Define visulization parameters
rainfall_vis = {
    'min': 0,
    'max': 100,
    'palette': ['white', 'blue']
}

# Add rainfall layer to map
Map.addLayer(rainfall, rainfall_vis, 'Rainfall')
Map

In [ ]:
# Download Rainfall Data as GeoTIFF
geemap.ee_export_image(rainfall, filename='rainfall_lagos.tif', scale=3000, region=aoi, file_per_band=False)

Load Land  Cover Data (ESA WorldCover 2020)

In [ ]:
# Load ESA WorldCover 2020 land cover dataset because land cover influences flood behavior. Urban areas reduce infiltration, while wetlands indicate flood
lulc = ee.ImageCollection('ESA/WorldCover/v100').select('Map').first().clip(aoi)

# Define visulization parameters
lulc_vis = {'min': 10, 'max': 100, 'palette': ['red', 'yellow', 'green']}

# Add land cover layer to map
Map.addLayer(lulc, lulc_vis, 'Land Cover')
Map


In [ ]:
# Download LULC Data as GeoTIFF
geemap.ee_export_image(lulc, filename='lulc_lagos.tif', scale=30, region=aoi, file_per_band=False)

Load Population Density Data (WorldPop)

In [ ]:
# Load WorldPop population dataset to assess population exosure to floods
population = ee.ImageCollection('WorldPop/GP/100m/pop')\
.filterBounds(aoi)\
.filterDate("2020-01-01", "2020-12-31")\
.mean()\
.clip(aoi)

# Define visualization parameters
population_vis = {
    'min': 0,
    'max': 1000,
    'palette': ['white', 'red']
}

# Add population layer to map
Map.addLayer(population, population_vis, 'Population Density')
Map

In [ ]:
# Download Population Data as GeoTIFF
geemap.ee_export_image(population, filename='population_lagos.tif', scale=100, region=aoi, file_per_band=False)

Perform Flood Risk Analysis

We will compute the Flood Risk Index(FRI) based on:
* Low-lying elevation (DEM)
* High precipitation levels (Rainfall)
* High urban development and wetlands (LULC)
* High population exposure (WorldPop)

# Normalize Data for Analysis


In [ ]:
# convert to numpy arrays for calculations
dem_array = geemap.ee_to_numpy(dem, scale=150)  # Use consistent scale
rainfall_array = geemap.ee_to_numpy(rainfall, scale=150)
lulc_array = geemap.ee_to_numpy(lulc, scale=150)
population_array = geemap.ee_to_numpy(population, scale=150)

#  get the minimum shape among all arrays
min_shape = min(dem_array.shape, rainfall_array.shape, lulc_array.shape, population_array.shape)

# resize all arrays to the minimum shape
dem_array = dem_array[:min_shape[0], :min_shape[1], :min_shape[2]]
rainfall_array = rainfall_array[:min_shape[0], :min_shape[1], :min_shape[2]]
lulc_array = lulc_array[:min_shape[0], :min_shape[1], :min_shape[2]]
population_array = population_array[:min_shape[0], :min_shape[1], :min_shape[2]]


# normalize the data to 0-1 range
norm_dem = (dem_array - dem_array.min()) / (dem_array.max() - dem_array.min())
norm_rainfall = (rainfall_array - rainfall_array.min()) / (rainfall_array.max() - rainfall_array.min())
norm_lulc = (lulc_array - lulc_array.min()) / (lulc_array.max() - lulc_array.min())
norm_population = (population_array - population_array.min()) / (population_array.max() - population_array.min())


# compute Flood Risk Index(FRI)
fri = (1 - norm_dem) * 0.4 + norm_rainfall * 0.3 + norm_lulc * 0.2 + norm_population * 0.1


In [ ]:
# Visualize Flood Risk
plt.figure(figsize=(10,6))
plt.imshow(fri, cmap='coolwarm')
plt.colorbar(label= 'Flood Risk Index (FRI)')
plt.title('Flood Risk Map of Lagos')
plt.show()

# Save and Export the Flood Risk Data

In [ ]:
# Reshape the fri array to have 3 dimensions if it has only 2
if fri.ndim == 2:
  fri = fri[:, :, np.newaxis] # Add a third dimension

# if fri still doesn't have 3 or 4 dimension after reshaping.
# It might contains multiple bands, in this case,  select  the firt band (or the relevant one)
if fri.ndim not in (3, 4):
  fri = fri[:, :, 0] # select the first band
  fri = fri[:, :, np.newaxis] # Add a third dimension

# Convert the single-channel fri array to a 3-channel RGB array by repeating the values across the 3 color channels.
# This assumes that the fri  values represent a grayscale image.
fri_rgb = np.repeat(fri, 3, axis=2)

# Now save the reshaped fri array as image
plt.imsave('flood_risk_lagos.png', fri_rgb, cmap='coolwarm')

In [ ]:
# Export as GeoTIFF
# Reshape fri to have expected dimensions: (height, width) for a single band
fri = fri.squeeze()

with rasterio.open('flood_risk_lagos.tif', 'w',
                    driver='GTiff',
                    height=fri.shape[0],
                    width=fri.shape[1],
                    count=1,
                    dtype=str(fri.dtype)
                    )as dst:
                    dst.write(fri, 1) # Write the reshaped array to the first band

